In [28]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import classification_report, confusion_matrix

In [153]:
df = pd.read_csv('https://s3.amazonaws.com/aml-sample-data/banking.csv', header=0)

In [154]:
df.rename(columns={'y':'willRespond'},inplace=True)

In [155]:
df.drop(['duration','contact','month','day_of_week','default','pdays',],axis=1,inplace=True) #Removing unnecessary columns

In [156]:
df.head()

,age,job,marital,education,housing,loan,campaign,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,willRespond
0,44,blue-collar,married,basic.4y,yes,no,1,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,1,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,yes,no,3,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,2,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,yes,no,1,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [157]:
le=preprocessing.LabelEncoder()

In [158]:
le.fit(df.poutcome)

LabelEncoder()

In [159]:
list(le.classes_)

['failure', 'nonexistent', 'success']

In [160]:
dle.classes_f.poutcome=le.transform(df.poutcome)

In [161]:

df.housing=le.fit_transform(df.housing)

In [162]:
le.classes_

array(['no', 'unknown', 'yes'], dtype=object)

In [163]:
df.loan=le.fit_transform(df.loan)

In [164]:
le.classes_

array(['no', 'unknown', 'yes'], dtype=object)

In [186]:
df.job=le.fit_transform(df.job)

In [187]:
le.classes_

array(['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
       'retired', 'self-employed', 'services', 'student', 'technician',
       'unemployed', 'unknown'], dtype=object)

In [189]:
df.marital=le.fit_transform(df.marital)

In [190]:
le.classes_

array(['divorced', 'married', 'single', 'unknown'], dtype=object)

In [191]:
df.education=le.fit_transform(df.education)

In [192]:
le.classes_

array(['basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate',
       'professional.course', 'university.degree', 'unknown'], dtype=object)

In [166]:
df.campaign.max()

56

In [14]:
list(le.classes_)

['basic.4y',
 'basic.6y',
 'basic.9y',
 'high.school',
 'illiterate',
 'professional.course',
 'university.degree',
 'unknown']

# Splitting and Creating a Logistic Regression Model

In [225]:
X = df.drop('willRespond',axis=1)
y = df.willRespond

In [226]:
smallX = X.loc[:,['age','job','marital','education','housing','loan','campaign']]
X = smallX

In [241]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=0,stratify=y) #70/30

In [86]:
num_trees = 100
sample_weight = np.array([6.5 if i == 1 else 1 for i in y_train]) # handling unbalanced 10:1 ratio

clf = RandomForestClassifier(n_estimators=num_trees , max_depth=2, n_jobs =4, random_state=0)

In [87]:
clf.fit(x_train, y_train,sample_weight= sample_weight)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [88]:
preds = clf.predict(x_test)

In [89]:
print("Evaluation report: \n\n%s" % classification_report(y_test, preds))

Evaluation report: 

             precision    recall  f1-score   support

          0       0.90      0.98      0.93     10965
          1       0.34      0.10      0.15      1392

avg / total       0.83      0.88      0.85     12357



In [90]:
confusion_matrix(y_test,preds)

array([[10698,   267],
       [ 1255,   137]])

In [39]:
#scoring with threshold
y_pred = pd.Series(final_scores).map(lambda x: 1 if x > 0.6 else 0)
np.mean(y_pred==y_test)

#print("Evaluation report: \n\n%s" % classification_report(y_test, prediction))

In [23]:
# # Ensembling

# x_train.loc[:,'target'] = y_train
# pos = x_train[x_train.target == 1]
# neg = x_train[x_train.target == 0]
# smp_size = int(pos.shape[0] * 0.9)

# #model training
# models = []
# for i in range(30):
#     pos_smp = pos.sample(n=smp_size)
#     neg_smp = neg.sample(n=smp_size)
#     full_smp = pos_smp.append(neg_smp)
#     x_smp = full_smp.drop('target',axis=1)
#     y_smp = full_smp.target
#     model=LogisticRegression(penalty='l2', max_iter=5)
#     model.fit(x_smp,y_smp)
#     models.append(model)
    
# def iterateModels(data):
#     scores = []
#     for mod in models:
#         scores.append(mod.predict_proba(data)[0][1])
#     return np.mean(scores)

# single = x_test.iloc[0,:].values
# single.shape
# s = single.reshape(1,7)
# models[0].predict_proba(s)[0][1]
# iterateModels(s)
#
# scores = np.zeros([x_test.shape[0],30])
# for i, model in enumerate(models):
#     scores[:,i] = model.predict_proba(x_test)[:,1]
# final_scores = scores.mean(axis=1)
# x_test.iloc[0,:].values
# final_scores[0]

### Convert function to pipeline transformer

In [95]:
single = x_test.iloc[0,:].values
s = single.reshape(1,7)

In [47]:
# transFunc = FunctionTransformer(iterateModels)
# myPipe = make_pipeline(transFunc)
# myPipe.transform(s)

In [91]:
myPipe = make_pipeline(clf)

In [97]:
myPipe.predict_proba(s)

array([[ 0.56217653,  0.43782347]])

### Store offline predictions for use in RShiny App 

In [229]:
test_preds = myPipe.predict_proba(x_test)
test_pred_class = myPipe.predict(x_test)

In [238]:
x_test.head()

,age,job,marital,education,housing,loan,campaign,pred_class,proba_no,proba_yes
39935,33,0,1,6,0,0,3,0,0.562177,0.437823
30199,32,4,1,6,0,0,2,0,0.539041,0.460959
17755,49,7,2,3,2,2,1,0,0.534492,0.465508
768,42,0,2,6,2,2,1,0,0.528961,0.471039
21784,54,0,1,6,2,0,1,0,0.551152,0.448848


In [242]:
x_test.loc[:,'actual_class'] = y_test
x_test.loc[:,'pred_class'] = test_pred_class
x_test.loc[:,'proba_no'] = test_preds[:,0]
x_test.loc[:,'proba_yes'] = test_preds[:,1]

/usr/local/src/conda3_runtime.v17/4.1.1/lib/python3.5/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/src/conda3_runtime.v17/4.1.1/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [243]:
x_test.head()

,age,job,marital,education,housing,loan,campaign,actual_class,pred_class,proba_no,proba_yes
39935,33,0,1,6,0,0,3,0,0,0.562177,0.437823
30199,32,4,1,6,0,0,2,0,0,0.539041,0.460959
17755,49,7,2,3,2,2,1,0,0,0.534492,0.465508
768,42,0,2,6,2,2,1,0,0,0.528961,0.471039
21784,54,0,1,6,2,0,1,0,0,0.551152,0.448848


In [244]:
from io import BytesIO  
import requests  
import json  
import pandas as pd

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print(resp2)


In [245]:
x_test.to_csv('Test_Scored.csv', index=False)

In [246]:
# The code was removed by DSX for sharing.

In [247]:
put_file(credentials_1, 'Test_Scored.csv')

<Response [201]>


In [207]:
test_preds

array([[ 0.56217653,  0.43782347],
       [ 0.53904056,  0.46095944],
       [ 0.53449183,  0.46550817],
       ..., 
       [ 0.52391123,  0.47608877],
       [ 0.60577129,  0.39422871],
       [ 0.51853334,  0.48146666]])

### Deploy!


In [50]:
#!pip install git+https://<username>:<token>@github.ibm.com/spark-service/project-lib.git --ignore-installed

In [117]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames
from project_lib import Project

In [118]:
# The code was removed by DSX for sharing.

In [119]:
proj = Project(sc, 'be994310-57f5-4351-9017-4dd6e0ce1817', 'p-1088294dc79b4eb3e43a160707efc5ead06433af')

In [120]:
#inserted by clicking project token 
proj_meta = proj.get_metadata()
#creds = [x for x in proj_meta['entity']['compute'] if x['type'] == 'machine_learning'][0]['credentials']
creds = [x for x in proj_meta['entity']['compute'] if x['type'] == 'machine_learning'][1]['credentials']
creds 

{'access_key': 'NFPpJtE9HQ4YdoUGt0VHw4Kl+2gk8Ga3LSeLilRqFCI+gl9ii76AJBDn71rBT/3kHxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv',
 'instance_id': '8a2b9093-833c-469c-b648-192bedbbbd4a',
 'password': 'e183a9fc-a67b-4aea-b787-59527a2da805',
 'url': 'https://ibm-watson-ml.mybluemix.net',
 'username': '4fae7652-8cf5-4fc5-a90b-6a80e8120cd0'}

In [121]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
username = creds['username']
password = creds['password']
instance_id = creds['instance_id']


In [122]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)
props = MetaProps({MetaNames.AUTHOR_NAME:"Greg Filla", MetaNames.AUTHOR_EMAIL:"gfilla@us.ibm.com"})

In [123]:
#model_artifact = MLRepositoryArtifact(myPipe, name="bank_lr_ensemble", meta_props=props)
model_artifact = MLRepositoryArtifact(myPipe, name="bank_rf", meta_props=props)

In [124]:
saved_model = ml_repository_client.models.save(model_artifact)

In [125]:
saved_model.meta.available_props()

dict_keys(['lastUpdated', 'version', 'authorEmail', 'trainingDataRef', 'runtime', 'pipelineVersionHref', 'modelVersionHref', 'creationTime', 'modelType', 'authorName'])

In [126]:
print("modelType: " + saved_model.meta.prop("modelType"))
print("runtime: " + saved_model.meta.prop("runtime"))
print("creationTime: " + str(saved_model.meta.prop("creationTime")))
print("modelVersionHref: " + saved_model.meta.prop("modelVersionHref"))

modelType: scikit-model-0.17
runtime: python-3.5
creationTime: 2017-09-23 17:48:13.007000+00:00
modelVersionHref: https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/a80515c8-1f8a-412f-8a1f-98126f64c15f/versions/0a00353a-d5de-432d-9e72-f4ed36379d53


In [127]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

In [128]:
print(loadedModelArtifact.name)
print(saved_model.uid)

bank_rf
a80515c8-1f8a-412f-8a1f-98126f64c15f


In [129]:
s

array([[33,  0,  1,  6,  0,  0,  3]])

In [130]:
predictions = loadedModelArtifact.model_instance().predict(s)

In [131]:
predictions

array([0])

In [132]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

In [133]:
endpoint_instance = service_path + "/v3/wml_instances/" + instance_id
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken} 

response_get_instance = requests.get(endpoint_instance, headers=header)
print(response_get_instance)
print(response_get_instance.text)

<Response [200]>
{"metadata":{"guid":"8a2b9093-833c-469c-b648-192bedbbbd4a","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a","created_at":"2017-09-23T17:46:03.389Z","modified_at":"2017-09-23T17:48:13.112Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models"},"usage":{"expiration_date":"2017-10-01T00:00:00.000Z","computation_time":{"limit":18000,"current":0},"model_count":{"limit":200,"current":1},"prediction_count":{"limit":5000,"current":0},"deployment_count":{"limit":5,"current":0}},"plan_id":"3f6acf43-ede8-413a-ac69-f8af3bb0cbfe","status":"Active","organization_guid":"59db2e81-0ab2-4abe-b2a1-f7da1b9ad1dc","region":"us-south","account":{"id":"501209268","name":"Greg Filla's Account","type":"TRIAL"},"owner":{"ibm_id":"310000V6NF","email":"gfilla@us.ibm.com","user_id":"c1eabbcb-3ff0-4f2a-b6b0-ed5c19df4f2f","country_code":"USA"

In [134]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print(endpoint_published_models)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models


In [135]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

print(response_get)
print(response_get.text)

<Response [200]>
{"count":1,"resources":[{"metadata":{"guid":"a80515c8-1f8a-412f-8a1f-98126f64c15f","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models/a80515c8-1f8a-412f-8a1f-98126f64c15f","created_at":"2017-09-23T17:48:13.007Z","modified_at":"2017-09-23T17:48:13.144Z"},"entity":{"runtime_environment":"python-3.5","author":{"name":"Greg Filla","email":"gfilla@us.ibm.com"},"name":"bank_rf","latest_version":{"url":"https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/a80515c8-1f8a-412f-8a1f-98126f64c15f/versions/0a00353a-d5de-432d-9e72-f4ed36379d53","guid":"0a00353a-d5de-432d-9e72-f4ed36379d53","created_at":"2017-09-23T17:48:13.144Z"},"model_type":"scikit-model-0.17","deployments":{"count":0,"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models/a80515c8-1f8a-412f-8a1f-98126f64c15f/deployments"}}}]}


In [139]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print(endpoint_deployments)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models/a80515c8-1f8a-412f-8a1f-98126f64c15f/deployments


In [140]:
payload_online = {"name": "Bank Loan Scoring v1_rf", "description": "Scoring bank applicants for loan approval", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

print(response_online)
print(response_online.text)

<Response [201]>
{"metadata":{"guid":"041a650f-a1c3-4f89-a6bc-43ddd57fe138","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models/a80515c8-1f8a-412f-8a1f-98126f64c15f/deployments/041a650f-a1c3-4f89-a6bc-43ddd57fe138","created_at":"2017-09-23T17:50:13.666Z","modified_at":"2017-09-23T17:50:15.100Z"},"entity":{"runtime_environment":"python-3.5","name":"Bank Loan Scoring v1_rf","scoring_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models/a80515c8-1f8a-412f-8a1f-98126f64c15f/deployments/041a650f-a1c3-4f89-a6bc-43ddd57fe138/online","description":"Scoring bank applicants for loan approval","published_model":{"author":{"name":"Greg Filla","email":"gfilla@us.ibm.com"},"name":"bank_rf","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8a2b9093-833c-469c-b648-192bedbbbd4a/published_models/a80515c8-1f8a-412f-8a1f-98126f64c15f","guid":"a80515c8-1f8a-412f-8a1f-98126f64c1

In [141]:
username

'4fae7652-8cf5-4fc5-a90b-6a80e8120cd0'

In [142]:
instance_id

'8a2b9093-833c-469c-b648-192bedbbbd4a'

In [143]:
password

'e183a9fc-a67b-4aea-b787-59527a2da805'

In [144]:
s

array([[33,  0,  1,  6,  0,  0,  3]])

In [148]:
df.willRespond.value_counts()

0    36548
1     4640
Name: willRespond, dtype: int64

In [146]:
x_train.head()

,age,job,marital,education,housing,loan,campaign
27598,51,9,0,3,0,0,3
31026,43,4,1,6,0,0,2
24524,31,4,1,6,0,0,3
3744,33,9,0,5,0,0,3
21460,25,0,2,2,0,0,29


In [173]:
predSer = pd.Series(preds) 
predSer.value_counts()

0    11953
1      404
dtype: int64

In [177]:
predSer.shape

(12357,)

In [178]:
x_test.shape

(12357, 7)

In [185]:
type(x_test)

pandas.core.frame.DataFrame

In [202]:
x_test.iloc[(predSer == 1).values, :]

,age,job,marital,education,housing,loan,campaign
22788,76,5,1,6,2,0,1
1651,79,5,1,0,0,2,1
19415,27,8,2,7,2,0,1
9086,75,5,1,2,0,0,1
35220,65,5,1,3,2,2,2
28059,61,1,1,0,2,0,1
29775,60,5,1,3,0,0,1
5241,72,5,1,0,0,0,1
7788,70,5,1,0,0,0,1
13983,69,5,1,3,2,0,1
